# Can the rosetta estimator distinguish between high stable and low stable?

Use 1enh and engineered version

In [1]:
from nomelt.thermo_estimation.rosetta import minimize_structures
from Bio.PDB import *
import nglview as nv
import ipywidgets
import Bio.PDB.PDBList
from nomelt.thermo_estimation.estimators import ESMFoldDGEstimator, ESMFoldDGArgs
from nomelt.thermo_estimation.rosetta import RosettaMinimizationParameters
parser = PDBParser()

PyRosetta-4 2023 [Rosetta PyRosetta4.Release.python39.ubuntu 2023.27+release.e3ce6ea9faf661ae8fa769511e2a9b8596417e58 2023-07-07T12:00:46] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.Release.python39.ubuntu r353 2023.27+release.e3ce6ea9faf e3ce6ea9faf661ae8fa769511e2a9b8596417e58 http://www.pyrosetta.org 2023-07-07T12:00:46
core.init: command: PyRosetta -ex1 -ex2aro -database /mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/pyrosetta-2023.27+release.e3ce6ea9faf-py3.9-linux-x86_64.egg/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=2004693461 seed_offset=0 real_seed=2004693461
basic.random.init_random_generator: RandomGenerator:init: Normal mode, seed=2004693461 RG_type=mt19937


[2023-07-27 15:12:28,712] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


2023-07-27 15:12:35.074277: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-27 15:12:37.035178: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
from Bio.PDB import *
import nglview as nv
import ipywidgets

In [3]:
import tempfile

In [4]:
tmpdir = tempfile.mkdtemp(dir='./tmp')

### First try the ESM estimator

In [5]:
rosetta_params = RosettaMinimizationParameters(update_pdb=False)
estimator_args = ESMFoldDGArgs(rosetta_params=rosetta_params)

In [6]:
ENH1 = "RPRTAFSSEQLARLKREFNENRYLTERRRQQLSSELGLNEAQIKIWFQNKRAKI"
_2P6J = "MKQWSENVEEKLKEFVKRHQRITQEELHQYAQRLGLNEEAIRQFFEEFEQRK"

In [7]:
estimator = ESMFoldDGEstimator(sequences= [ENH1, _2P6J], ids = ['1enh', '2p6j'])

In [8]:
estimator.run()

100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.88s/it]


core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 985 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 0.702236 seconds.
core.import_pose.import_pose: File './tmp/ESMFoldDGEstimator_structures/1enh.pdb' automatically determined to be of type PDB
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue ILE:CtermProteinFull 54
core.scoring.etable: Starting energy table calculation
core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: Finished calculating energy tables.
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBPoly1D.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBFadeIntervals.csv
basic.io.data

{'1enh': -132.44393295526166, '2p6j': -110.2349447847906}

The engineered variant is higher in energy than the WT.... not good. Maybe the structure is bungo

In [9]:
_2P6J_struc = parser.get_structure('2P6J',estimator.pdb_files_dict['2p6j'])

/mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'PARENT' at line 468
  warnings.warn(
/mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 470
  warnings.warn(


In [10]:
nv.show_biopython(_2P6J_struc, gui=True)

NGLWidget()

ESM thinks the engineered variant is not a 3hb

### Now do jsut rosetta on PDB structs, get the two structure for rosetta from PDB

In [21]:
pdblist = Bio.PDB.PDBList()

In [24]:
pdblist.retrieve_pdb_file('1ENH', pdir=tmpdir, file_format='pdb')

'./tmp/tmp6b4ggwfx/pdb1enh.ent'

In [25]:
pdblist.retrieve_pdb_file('2P6J', pdir=tmpdir, file_format='pdb')

'./tmp/tmp6b4ggwfx/pdb2p6j.ent'

#### the engineered variant is a solvent model, let's get only one structure

In [27]:
_2P6J_struc = parser.get_structure('2P6J',tmpdir+'/pdb2p6j.ent')

In [64]:
model = next(_2P6J_struc.get_models())

In [65]:
nv.show_biopython(model, gui=True)

NGLWidget()

In [66]:
len([a for a in model.get_atoms()])

919

In [67]:
from Bio.PDB.PDBIO import PDBIO

In [68]:
io=PDBIO()
io.set_structure(model)
io.save(tmpdir+'/2p6j_single.pdb')

### Run rosetta on the pdbs

In [51]:
minimize_structures([tmpdir+'/pdb1enh.ent', tmpdir+'/2p6j_single.pdb'])

PyRosetta-4 2023 [Rosetta PyRosetta4.Release.python39.ubuntu 2023.27+release.e3ce6ea9faf661ae8fa769511e2a9b8596417e58 2023-07-07T12:00:46] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.Release.python39.ubuntu r353 2023.27+release.e3ce6ea9faf e3ce6ea9faf661ae8fa769511e2a9b8596417e58 http://www.pyrosetta.org 2023-07-07T12:00:46
core.init: command: PyRosetta -ex1 -ex2aro -database /mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/pyrosetta-2023.27+release.e3ce6ea9faf-py3.9-linux-x86_64.egg/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=1353088276 seed_offset=0 real_seed=1353088276
basic.random.init_random_generator: RandomGenerator:init: Normal mode, seed=1353088276 RG_type=mt19937
core.che

[18.660016044232023, 173.3902176637841]

### Manual rosetta

#### 1enh

In [70]:
import pyrosetta # Local import
import pyrosetta.io
from pyrosetta.rosetta.protocols.minimization_packing import MinMover
pyrosetta.init()


PyRosetta-4 2023 [Rosetta PyRosetta4.Release.python39.ubuntu 2023.27+release.e3ce6ea9faf661ae8fa769511e2a9b8596417e58 2023-07-07T12:00:46] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.Release.python39.ubuntu r353 2023.27+release.e3ce6ea9faf e3ce6ea9faf661ae8fa769511e2a9b8596417e58 http://www.pyrosetta.org 2023-07-07T12:00:46
core.init: command: PyRosetta -ex1 -ex2aro -database /mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/pyrosetta-2023.27+release.e3ce6ea9faf-py3.9-linux-x86_64.egg/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=945069454 seed_offset=0 real_seed=945069454
basic.random.init_random_generator: RandomGenerator:init: Normal mode, seed=945069454 RG_type=mt19937


In [71]:
# Create a pose from the PDB file
pose=pyrosetta.io.pose_from_pdb(tmpdir+'/pdb1enh.ent')
# Create a score function
scorefxn = pyrosetta.create_score_function('ref2015')
# Add constraints to the score function, if requested
scorefxn(pose)

core.import_pose.import_pose: File './tmp/tmp6b4ggwfx/pdb1enh.ent' automatically determined to be of type PDB
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue ILE:CtermProteinFull 54


18.660016044232023

In [72]:
# Setup the MinMover
min_mover = MinMover()
mm = pyrosetta.MoveMap()
mm.set_bb(True)
mm.set_chi(True)
mm.set_jump(True)
min_mover.score_function(scorefxn)
min_mover.movemap(mm)
min_mover.min_type('lbfgs_armijo_nonmonotone')
min_mover.tolerance(.0001)

# Apply the MinMover to the pose
min_mover.apply(pose)

In [73]:
scorefxn(pose)

-122.29198104266534

#### engineered one

In [80]:
# Create a pose from the PDB file
pose=pyrosetta.io.pose_from_pdb(tmpdir+'/2p6j_single.pdb')
# Create a score function
scorefxn = pyrosetta.create_score_function('ref2015')
# Add constraints to the score function, if requested
scorefxn(pose)

core.import_pose.import_pose: File './tmp/tmp6b4ggwfx/2p6j_single.pdb' automatically determined to be of type PDB
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 3 atoms at position 1 in file ./tmp/tmp6b4ggwfx/2p6j_single.pdb. Best match rsd_type:  MET:NtermProteinFull


173.3902176637841

In [81]:
# Setup the MinMover
min_mover = MinMover()
mm = pyrosetta.MoveMap()
mm.set_bb(True)
mm.set_chi(True)
mm.set_jump(True)
min_mover.score_function(scorefxn)
min_mover.movemap(mm)
min_mover.min_type('lbfgs_armijo_nonmonotone')
min_mover.tolerance(.0001)

# Apply the MinMover to the pose
min_mover.apply(pose)

In [82]:
scorefxn(pose)

-66.21308110919267

The engineered variant seems to have a more positive G unfolding...

### try consensus engineered one.

These sequences are highly similar, so are a good test of how well this estrimator could distinguish in the mutation regime we are working in.

In [11]:
rosetta_params = RosettaMinimizationParameters(update_pdb=False)
estimator_args = ESMFoldDGArgs(rosetta_params=rosetta_params)

In [12]:
concensus = "RRKRTTFTKEQLEELEELFEKNRYPSAEEREELAKKLGLTERQVKVWFQNRRAKEKK"

In [13]:
enh1 = "RPRTAFSSEQLARLKREFNENRYLTERRRQQLSSELGLNEAQIKIWFQNKRAKI"

In [14]:
estimator = ESMFoldDGEstimator(sequences = [enh1, concensus], ids=['wt', 'concensus'])

In [15]:
estimator.run()

100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.27s/it]


core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 985 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 0.690042 seconds.
core.import_pose.import_pose: File './tmp/ESMFoldDGEstimator_structures/wt.pdb' automatically determined to be of type PDB
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue ILE:CtermProteinFull 54
core.scoring.etable: Starting energy table calculation
core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: Finished calculating energy tables.
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBPoly1D.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBFadeIntervals.csv
basic.io.databa

{'wt': -130.62494306511954, 'concensus': -140.3217549930466}

It seems like it can distinguish between them. Let's make sure the structures are good.

In [16]:
parser = PDBParser()

In [17]:
estimator.pdb_files_dict

{'wt': './tmp/ESMFoldDGEstimator_structures/wt.pdb',
 'concensus': './tmp/ESMFoldDGEstimator_structures/concensus.pdb'}

In [18]:
wt_struct = parser.get_structure('wt', estimator.pdb_files_dict['wt'])
conc_struct = parser.get_structure('conc', estimator.pdb_files_dict['concensus'])

/mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'PARENT' at line 469
  warnings.warn(
/mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 471
  warnings.warn(
/mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 504
  warnings.warn(


In [19]:
nv.show_biopython(wt_struct, gui=True)

NGLWidget()

In [20]:
nv.show_biopython(conc_struct, gui=True)

NGLWidget()